In [1]:
print("Hello World")

Hello World


In [3]:
import glob
import os

from liftout.model.utils import *
from liftout import model
from liftout.detection.detection import *
from liftout.detection.DetectionModel import *
from liftout.detection.utils import *

import matplotlib.pyplot as plt
import numpy as np

from PIL import Image
from random import shuffle

# print(os.path.dirname(model.__file__))

model_path = os.path.dirname(model.__file__)
model_filenames = glob.glob(
    os.path.join(model_path, "models", "*.pt")
)


image_fnames = glob.glob(
    os.path.join(model_path, "data/data/train/raw/000001*.tif")
)

shuffle(image_fnames)
model_filenames = sorted([fname for fname in model_filenames if any(match in fname for match in ["boost_n05", "n10", "baseline"])])
# any(x in a_string for x in matches)("boost" in fname) or ("n10" in fname)])

# there is a smarter way to do this (just order them correctly)
model_filenames.insert(1, model_filenames[-2])
model_filenames.insert(2, model_filenames[-1])
model_filenames.pop()
model_filenames.pop()
model_filenames.pop()
print(model_filenames)


['\\\\ad.monash.edu\\home\\user007\\prcle2\\documents\\demarco\\autoliftout\\liftout\\model\\models\\baseline_model.pt', '\\\\ad.monash.edu\\home\\user007\\prcle2\\documents\\demarco\\autoliftout\\liftout\\model\\models\\fresh_full_n10.pt', '\\\\ad.monash.edu\\home\\user007\\prcle2\\documents\\demarco\\autoliftout\\liftout\\model\\models\\n10_model.pt', '\\\\ad.monash.edu\\home\\user007\\prcle2\\documents\\demarco\\autoliftout\\liftout\\model\\models\\boost_n05_model.pt']


In [4]:



N_IMAGES = 20
EXP_NAME = input("Enter a name for the experiment")
EXP_PATH = os.path.join("eval", EXP_NAME)
os.makedirs(
    EXP_PATH, exist_ok=True)
    
image_fnames = image_fnames[:N_IMAGES]

exp_array = None
for weights_file in model_filenames:
    # load detector
    detector = Detector(weights_file=weights_file)
    detector.detection_model.model.eval()

    img_array = None

    for fname in image_fnames:
        # load image from file
        img = load_image_from_file(fname)
    
        # model inference
        mask = detector.detection_model.model_inference(img)

        # individual detection modes
        lamella_mask, lamella_idx = extract_class_pixels(mask, color=(255, 0, 0)) # red
        needle_mask, needle_idx = extract_class_pixels(mask, color=(0, 255, 0)) # green

        feature_1_px, lamella_centre_detection = detect_lamella_centre(img, mask) # lamella_centre
        feature_2_px, needle_tip_detection = detect_needle_tip(img, mask)
        feature_3_px, lamella_edge_detection = detect_lamella_edge(img, mask)

        mask_combined = draw_two_features(mask, feature_1_px, feature_2_px)
        img_blend = draw_overlay(img, mask_combined, show=False)
        
        if img_array is None:
            img_array = np.array(img_blend)

        else:
            img_array = np.hstack((img_array, np.array(img_blend)))

    print(f"saving: {os.path.basename(weights_file)}")
    img_pil = Image.fromarray(img_array)
    img_pil.save(os.path.join(EXP_PATH, f"{os.path.basename(weights_file)}.png"))

    if exp_array is None:
        exp_array = img_array
    else:
        exp_array = np.vstack((exp_array, img_array))

exp_img_pil = Image.fromarray(exp_array)
exp_img_pil.save(os.path.join(EXP_PATH,f"experiment.png"))
print("Finished.")

saving: baseline_model.pt
saving: fresh_full_n10.pt
saving: n10_model.pt
saving: boost_n05_model.pt
Finished.
